# Wikipedia Retriever

In [3]:
from langchain_community.retrievers import WikipediaRetriever

In [5]:
# Initialize the retriever 
retriever = WikipediaRetriever(top_k_results=2, lang='en')

In [6]:
# Define your query
query = 'the geopolitical history of india and pakistan from the perspective of a chinese'

# Get relevant wikipedia docs
docs = retriever.invoke(query)

In [7]:
docs

[Document(metadata={'title': 'United States aid to Pakistan', 'summary': 'The United States has been providing military aid and economic assistance to Pakistan for various purposes since 1948. In 2017, the U.S. stopped military aid to Pakistan, which was about US$2 billion per year. With U.S. military assistance suspended in 2018 and civilian aid reduced to about $300 million for 2022, Pakistani authorities have turned to other countries for help.\n\n', 'source': 'https://en.wikipedia.org/wiki/United_States_aid_to_Pakistan'}, page_content="The United States has been providing military aid and economic assistance to Pakistan for various purposes since 1948. In 2017, the U.S. stopped military aid to Pakistan, which was about US$2 billion per year. With U.S. military assistance suspended in 2018 and civilian aid reduced to about $300 million for 2022, Pakistani authorities have turned to other countries for help.\n\n\n== History ==\nFrom 1947 to 1958, under civilian leadership, the United

In [8]:
# Print retrieved document
for i, doc in enumerate(docs):
    print(f'\n----Result---- {i+1} -----')
    print(f'Content: \n {doc.page_content}....')


----Result---- 1 -----
Content: 
 The United States has been providing military aid and economic assistance to Pakistan for various purposes since 1948. In 2017, the U.S. stopped military aid to Pakistan, which was about US$2 billion per year. With U.S. military assistance suspended in 2018 and civilian aid reduced to about $300 million for 2022, Pakistani authorities have turned to other countries for help.


== History ==
From 1947 to 1958, under civilian leadership, the United States provided Pakistan with modest economic aid and limited military assistance. During this period, Pakistan became a member of the South East Asian Treaty Organization (SEATO) and the Central Treaty Organization (CENTO), after a Mutual Defence Assistance Agreement signed in May 1954, which facilitated increased levels of both economic and military aid from the U.S.
In 1958, Ayub Khan led Pakistan's first military coup, becoming Chief Martial Law Administrator (CMLA) and later President until 1969. During 

# Vector Store Retriever

In [9]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

In [10]:
# Step 1 Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [11]:
from dotenv import load_dotenv


# Initialize embedding models
embedding_model = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

# Create chroma vectorstore in a retriever
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name='My_collection'
)

In [13]:
# Create a vector store into a retriever
retriever = vector_store.as_retriever(search_kwargs={'k':2})

In [14]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [15]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


# MMR

In [17]:
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [23]:
from langchain_classic.vectorstores import FAISS

# Initialize googe embeddings
embedding_model = GoogleGenerativeAIEmbeddings(model='gemini-embedding-001')

# Create an FAISS vector store 
vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embedding_model
)

In [25]:
# Enable MMR in retriever
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k':3, 'lambda_muk':0.5} #k = top results, lambda_mult = relevance-diversity balance
)

In [26]:
query = 'What is langchain'
results = retriever.invoke(query)

In [27]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain helps developers build LLM applications easily.

--- Result 2 ---
Embeddings convert text into high-dimensional vectors.

--- Result 3 ---
Chroma is a vector database optimized for LLM-based search.


# Multiquery Retriever


In [28]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [29]:
# Initialize embeddings
embedding_model = GoogleGenerativeAIEmbeddings(model='gemini-embedding-001')

# Create FAISS vectore store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

In [30]:
# Create retrievers
similarity_retrievers = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k':5})

In [36]:
from langchain_classic.retrievers import MultiQueryRetriever
from langchain_google_genai  import GoogleGenerativeAI
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={'k':5}),
    llm = GoogleGenerativeAI(model='gemini-2.5-flash')
)

In [37]:
# Query 
query = "How to improve energy levels and maintain balance?"

In [38]:
# Retrieve results 
similarity_results = similarity_retrievers.invoke(query)
multi_query_results = multi_query_retriever.invoke(query)

In [39]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multi_query_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 4 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 5 ---
Regular walking boosts heart health and can reduce symptoms of depression.
******************************************************************************************************************************************************

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 4 ---
The solar energy system in modern homes helps balance el

# ContextualCompressionRetriever

In [40]:
from langchain_classic.retrievers import ContextualCompressionRetriever

In [41]:
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [ ]:
# Create FAISS vector store from the documents
vectorstore = FAISS.from_documents(docs, embedding_model)

In [43]:
base_retriever = vectorstore.as_retriever(search_kwargs={'k':5})

In [46]:
# Set up the compressor using LLM
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
llm = GoogleGenerativeAI(model='gemini-2.5-flash')
compressor = LLMChainExtractor.from_llm(llm)

In [47]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [48]:
# Query the retiever 
query = 'What is photosynthesis'
compressed_result = compression_retriever.invoke(query)

In [50]:
for i, doc in enumerate(compressed_result):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
Photosynthesis does not occur in animal cells.

--- Result 3 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
